# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [77]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
sport = 'NHL'

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    sport = 'NHL'
    
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df[nhl_df['team'] != nhl_df['W']]
    nhl_df['team'] = nhl_df['team'].str.extract('([\w. ]*)')
    nhl_df[sport] = nhl_df['team'].str.extract(r'([^ ]*$)')
    nhl_df = nhl_df.reset_index(drop=True)

    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
    cities[sport] = cities[sport].str.extract('([\w ]*)')
    cities[sport] = cities[sport].str.extract(r'([^ ]*$)')
    cities = cities[cities[sport] != '']
    cities.dropna(inplace=True)
    cities = cities.reset_index(drop=True)
    cities = cities.sort_values('Metropolitan area')
    cities['Population'] = cities['Population'].astype('float64')
    population_by_region = list(cities['Population'])


    nhl_df = nhl_df.merge(cities[['Metropolitan area', sport]], on=sport, how='left')
    nhl_df.loc[[12,14,15], 'Metropolitan area'] = 'New York City'
    nhl_df.fillna('Los Angeles', inplace=True)
    nhl_df['W'] = nhl_df['W'].astype('float64')
    nhl_df['L'] = nhl_df['L'].astype('float64')
    nhl_df = nhl_df.groupby('Metropolitan area').agg({'W':np.average, 'L': np.average})
    nhl_df['W/L%'] = (nhl_df['W']/(nhl_df['W'] + nhl_df['L'])).astype('float64')
    win_loss_by_region = list(nhl_df['W/L%'])

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

In [78]:
nhl_correlation()

(0.012308996455744264, 0.9504308637909502)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [310]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
sport = 'NBA'

def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    sport = 'NBA'
    
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df['team'].str.extract('([\w. ]*)')
    nba_df[sport] = nba_df['team'].str.extract(r'([^ ]*$)')
    nba_df = nba_df.reset_index(drop=True)
    
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
    cities[sport] = cities[sport].str.extract('([\w ]*)')
    cities[sport] = cities[sport].str.extract(r'([^ ]*$)')
    cities = cities[cities[sport] != '']
    cities.dropna(inplace=True)
    cities = cities.reset_index(drop=True)
    cities = cities.sort_values('Metropolitan area')
    cities['Population'] = cities['Population'].astype('float64')
    population_by_region = list(cities['Population'])
    
    
    nba_df = nba_df.merge(cities[['Metropolitan area', sport]], on=sport, how='left')
    nba_df.loc[[10,11], 'Metropolitan area'] = 'New York City'
    nba_df.fillna('Los Angeles', inplace=True)
    nba_df['W/L%'] = nba_df['W/L%'].astype('float64')
    nba_df = nba_df.groupby('Metropolitan area').agg({'W/L%':np.average})
    win_loss_by_region = list(nba_df['W/L%'])

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [311]:
nba_correlation()

(-0.17636350642182935, 0.36932106185547353)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [278]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
sport = 'MLB'

def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    sport = 'MLB'
    
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df['team'] = mlb_df['team'].str.extract('([\w. ]*)')
    mlb_df[sport] = mlb_df['team'].str.extract('([A-Z]{0,2}[a-z0-9]*\ Sox$|[A-Z]{0,2}[a-z0-9]*\ Jays$|[^ ]*$)')
    mlb_df = mlb_df.reset_index(drop=True)
    
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
    cities[sport] = cities[sport].str.extract('([\w ]*)')
    cities['Population'] = cities['Population'].astype('float64')
    population_by_region = list(cities[cities[sport] != ''].sort_values('Metropolitan area')['Population'])
    team = cities[sport].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable'])
    team = team[team['value'] != '']
    team = team.reset_index(drop=True)
    team = team.sort_values('Metropolitan area')
    cities = cities.sort_values('Metropolitan area')
    cities = cities.merge(team, on='Metropolitan area', how='left')
    cities[sport] = cities['value']
    cities = cities.drop(columns=['value'])
    cities = cities[cities['MLB'].isna() == False]
    
    mlb_df = mlb_df.merge(cities[['Metropolitan area', sport]], on=sport, how='left')
    mlb_df = mlb_df.groupby('Metropolitan area').agg({'W-L%':np.average})
    mlb_df['W-L%'] = mlb_df['W-L%'].astype('float64')
    win_loss_by_region = list(mlb_df['W-L%'])

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [279]:
mlb_correlation()

(0.15003737475409495, 0.46442827201123427)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [50]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
sport = 'NFL'

def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    sport = 'NFL'
    
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df[nfl_df['team'] != nfl_df['W']]
    nfl_df['team'] = nfl_df['team'].str.extract('([\w. ]*)')
    nfl_df[sport] = nfl_df['team'].str.extract('([A-Z]{0,2}[a-z0-9]*\ Sox$|[A-Z]{0,2}[a-z0-9]*\ Jays$|[^ ]*$)')
    nfl_df = nfl_df.reset_index(drop=True)
    
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
    cities[sport] = cities[sport].str.extract('([\w ]*)')
    cities['Population'] = cities['Population'].astype('float64')
    population_by_region = list(cities[cities[sport] != ''].sort_values('Metropolitan area')['Population'])
    team = cities[sport].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable'])
    team = team[team['value'] != '']
    team = team.reset_index(drop=True)
    team = team.sort_values('Metropolitan area')
    cities = cities.sort_values('Metropolitan area')
    cities = cities.merge(team, on='Metropolitan area', how='left')
    cities[sport] = cities['value']
    cities = cities.drop(columns=['value'])
    cities = cities[cities[sport].isna() == False]
    
    nfl_df = nfl_df.merge(cities[['Metropolitan area', sport]], on=sport, how='left')
    nfl_df['W-L%'] = nfl_df['W-L%'].astype('float64')
    nfl_df = nfl_df.groupby('Metropolitan area').agg({'W-L%':np.average})
    win_loss_by_region = list(nfl_df['W-L%'])

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [51]:
nfl_correlation()

(0.004282141436393022, 0.9824114740736553)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [72]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
sports = ['NFL', 'NBA', 'NHL', 'MLB']

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [66]:
cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})

    team = cities[sport].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable'])
    team = team[team['value'] != '']
    team = team.reset_index(drop=True)
    team = team.sort_values('Metropolitan area')
    cities = cities.sort_values('Metropolitan area')
    cities = cities.merge(team, on='Metropolitan area', how='left')
    cities[sport] = cities['value']
    cities = cities.drop(columns=['value'])
    cities = cities[cities[sport].isna() == False]

cities.drop_duplicates(subset=None, keep='first', inplace=True)

In [135]:
def nhl_win_loss(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    sport = 'NHL'
    
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df[nhl_df['team'] != nhl_df['W']]
    nhl_df['team'] = nhl_df['team'].str.extract('([\w. ]*)')
    nhl_df[sport] = nhl_df['team'].str.extract(r'([^ ]*$)')
    nhl_df = nhl_df.reset_index(drop=True)

    cities[sport] = cities[sport].str.extract('([\w ]*)')
    cities[sport] = cities[sport].str.extract(r'([^ ]*$)')
    cities = cities[cities[sport] != '']
    cities.dropna(inplace=True)
    cities = cities.reset_index(drop=True)
    cities = cities.sort_values('Metropolitan area')


    nhl_df = nhl_df.merge(cities[['Metropolitan area', sport]], on=sport, how='left')
    nhl_df.loc[[12,14,15], 'Metropolitan area'] = 'New York City'
    nhl_df.fillna('Los Angeles', inplace=True)
    nhl_df['W'] = nhl_df['W'].astype('float64')
    nhl_df['L'] = nhl_df['L'].astype('float64')
    nhl_df = nhl_df.groupby('Metropolitan area').agg({'W':np.average, 'L': np.average})
    nhl_df['W/L%'] = (nhl_df['W']/(nhl_df['W'] + nhl_df['L'])).astype('float64')
    nhl_df = nhl_df.reset_index().drop(columns=['W', 'L']).rename(columns={'W/L%':'NHL'})
    
    return nhl_df

df_nhl = nhl_win_loss()

In [136]:
def nba_win_loss():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    sport = 'NBA'
    
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df['team'].str.extract('([\w. ]*)')
    nba_df[sport] = nba_df['team'].str.extract(r'([^ ]*$)')
    nba_df = nba_df.reset_index(drop=True)
    
    cities[sport] = cities[sport].str.extract('([\w ]*)')
    cities[sport] = cities[sport].str.extract(r'([^ ]*$)')
    cities = cities[cities[sport] != '']
    cities.dropna(inplace=True)
    cities = cities.reset_index(drop=True)
    cities = cities.sort_values('Metropolitan area')    
    
    nba_df = nba_df.merge(cities[['Metropolitan area', sport]], on=sport, how='left')
    nba_df.loc[[10,11], 'Metropolitan area'] = 'New York City'
    nba_df.fillna('Los Angeles', inplace=True)
    nba_df['W/L%'] = nba_df['W/L%'].astype('float64')
    nba_df = nba_df.groupby('Metropolitan area').agg({'W/L%':np.average}).reset_index().rename(columns={'W/L%':'NBA'})

    return nba_df

df_nba = nba_win_loss()

In [137]:
def mlb_win_loss(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    sport = 'MLB'
    
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df['team'] = mlb_df['team'].str.extract('([\w. ]*)')
    mlb_df[sport] = mlb_df['team'].str.extract('([A-Z]{0,2}[a-z0-9]*\ Sox$|[A-Z]{0,2}[a-z0-9]*\ Jays$|[^ ]*$)')
    mlb_df = mlb_df.reset_index(drop=True)
    
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
    cities[sport] = cities[sport].str.extract('([\w ]*)')
    team = cities[sport].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable'])
    team = team[team['value'] != '']
    team = team.reset_index(drop=True)
    team = team.sort_values('Metropolitan area')
    cities = cities.sort_values('Metropolitan area')
    cities = cities.merge(team, on='Metropolitan area', how='left')
    cities[sport] = cities['value']
    cities = cities.drop(columns=['value'])
    cities = cities[cities['MLB'].isna() == False]
    
    mlb_df = mlb_df.merge(cities[['Metropolitan area', sport]], on=sport, how='left')
    mlb_df['W-L%'] = mlb_df['W-L%'].astype('float64')
    mlb_df = mlb_df.groupby('Metropolitan area').agg({'W-L%':np.average}).reset_index().rename(columns={'W-L%': 'MLB'})
    return mlb_df

df_mlb = mlb_win_loss()

In [138]:
def nfl_win_loss(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    sport = 'NFL'
    
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df[nfl_df['team'] != nfl_df['W']]
    nfl_df['team'] = nfl_df['team'].str.extract('([\w. ]*)')
    nfl_df[sport] = nfl_df['team'].str.extract('([A-Z]{0,2}[a-z0-9]*\ Sox$|[A-Z]{0,2}[a-z0-9]*\ Jays$|[^ ]*$)')
    nfl_df = nfl_df.reset_index(drop=True)
    
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
    cities[sport] = cities[sport].str.extract('([\w ]*)')
    team = cities[sport].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable'])
    team = team[team['value'] != '']
    team = team.reset_index(drop=True)
    team = team.sort_values('Metropolitan area')
    cities = cities.sort_values('Metropolitan area')
    cities = cities.merge(team, on='Metropolitan area', how='left')
    cities[sport] = cities['value']
    cities = cities.drop(columns=['value'])
    cities = cities[cities[sport].isna() == False]
    
    nfl_df = nfl_df.merge(cities[['Metropolitan area', sport]], on=sport, how='left')
    nfl_df['W-L%'] = nfl_df['W-L%'].astype('float64')
    nfl_df = nfl_df.groupby('Metropolitan area').agg({'W-L%':np.average}).reset_index().rename(columns={'W-L%':'NFL'})

    return nfl_df

df_nfl = nfl_win_loss()

In [139]:
import functools

dfs = [df_nhl, df_nba, df_mlb, df_nfl]

df_final = functools.reduce(lambda left,right: pd.merge(left,right,on='Metropolitan area', how='outer'), dfs)

In [141]:
df = df_final.dropna(thresh=3)

In [156]:
df2 = df.drop(['W/L% MLB', 'W/L% NFL'], axis=1).dropna()
df2

,Metropolitan area,W/L% NHL,W/L% NBA
0,Boston,0.714286,0.6710
3,Chicago,0.458333,0.3290
5,Dallas–Fort Worth,0.567568,0.2930
6,Denver,0.589041,0.5610
7,Detroit,0.434783,0.4760
10,Los Angeles,0.622378,0.4695
11,Miami–Fort Lauderdale,0.594595,0.5370
12,Minneapolis–Saint Paul,0.633803,0.5730
15,New York City,0.518349,0.3475
17,Philadelphia,0.617647,0.6340


In [159]:
stats.ttest_rel(df2['W/L% NHL'], df2['W/L% NBA'])

Ttest_relResult(statistic=2.5905361064540027, pvalue=0.02240459747249962)